In [4]:
import numpy as np 
import pandas as pd
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import pandas as pd
import pickle
import ast
#pd.set_option('display.max_columns', None)
#pd.set_option('display.width', 500)
#pd.set_option('max_colwidth', 150) #Kolondaki verilerin tüm içeriğini görebilmek için
#pd.set_option('display.expand_frame_repr', False)
#from sklearn.feature_extraction.text import TfidfVectorizer
#from sklearn.metrics.pairwise import cosine_similarity

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

warnings.simplefilter(action='ignore', category=Warning)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/itemBasedRecom/final_ingredients.csv
/kaggle/input/itemBasedRecom/final_repices_all.csv
/kaggle/input/itemBasedRecom/final_repices.csv
/kaggle/input/food-com-recipes-and-user-interactions/RAW_interactions.csv
/kaggle/input/food-com-recipes-and-user-interactions/ingr_map.pkl
/kaggle/input/food-com-recipes-and-user-interactions/PP_recipes.csv
/kaggle/input/food-com-recipes-and-user-interactions/RAW_recipes.csv
/kaggle/input/food-com-recipes-and-user-interactions/interactions_train.csv
/kaggle/input/food-com-recipes-and-user-interactions/interactions_test.csv
/kaggle/input/food-com-recipes-and-user-interactions/PP_users.csv
/kaggle/input/food-com-recipes-and-user-interactions/interactions_validation.csv


In [3]:
#get raw_interactions dataframe
raw_interactions=pd.read_csv('/kaggle/input/food-com-recipes-and-user-interactions/RAW_interactions.csv')
raw_interactions.head()

,user_id,recipe_id,date,rating,review
0,38094,40893,2003-02-17,4,Great with a salad. Cooked on top of stove for...
1,1293707,40893,2011-12-21,5,"So simple, so delicious! Great for chilly fall..."
2,8937,44394,2002-12-01,4,This worked very well and is EASY. I used not...
3,126440,85009,2010-02-27,5,I made the Mexican topping and took it to bunk...
4,57222,85009,2011-10-01,5,"Made the cheddar bacon topping, adding a sprin..."


In [5]:
#get the final recipe_ids used in content based recommender
final_recipes=pd.read_csv('/kaggle/input/itemBasedRecom/final_repices.csv')
final_recipes.head()

,id,name
0,25775,how i got my family to eat spinach spinach ca...
1,22526,land of nod cinnamon buns
2,74805,never weep whipped cream
3,83950,real italian bolognese sauce
4,155959,ant kelly s london broil marinade


In [6]:
#filter the interactions with the recipe_id's used in content based recommender
filtered_interactions=raw_interactions[raw_interactions["recipe_id"].isin(final_recipes.id)]

In [7]:
#user & recipe id pivot
user_recipe_df = filtered_interactions.pivot_table(index =["user_id"], columns = ["recipe_id"], values = "rating")

In [56]:
#Test with a recipe id
recipe_id=456
recom_df=pd.DataFrame({}, columns=['recipe_id','recom_ids','recom_corrs'])
recipe_id_filter=user_recipe_df[recipe_id]
recom = user_recipe_df.corrwith(recipe_id_filter).sort_values(ascending=False).head(6).to_frame('corr_value')
recom_ids=list(recom.index)

recom_names=[]
for r_id in recom_list:
    recom_names.append(final_recipes[final_recipes.id == r_id]['name'].values[0])

corr_list=list(recom.corr_value)


recom_df = recom_df.append({'recipe_id': recipe_id, 'recom_ids': recom_ids,'recom_names': recom_names, 'recom_corrs': corr_list}, ignore_index=True)

# Now what needs to be done is to collect top 5 recipes per recipe_id to recommend and save it in a dataframe 

In [61]:
#Create a blank df to save results per recipe
recom_df=pd.DataFrame({}, columns=['recipe_id','recom_ids','recom_names','recom_corrs'])

for recipe_id in list(final_recipes.id):
    #find the series object for relevant recipe_id in the user_recipe_df
    recipe_id_filter=user_recipe_df[recipe_id]
    
    #find the top 6 recommended recipe_id's and their corr values
    recom = user_recipe_df.corrwith(recipe_id_filter).sort_values(ascending=False).head(6).to_frame('corr_value')
    recom_ids=list(recom.index)
    corr_list=list(recom.corr_value)

    recom_names=[] #get the names of recommended recipe_id's
    for r_id in recom_ids:
        recom_names.append(final_recipes[final_recipes.id == r_id]['name'].values[0])

    #append the results to the dataframe
    recom_df = recom_df.append({'recipe_id': recipe_id, 'recom_ids': recom_ids,'recom_names': recom_names, 'recom_corrs': corr_list}, ignore_index=True)

In [62]:
#export recommendations as a csv file 
recom_df.to_csv('ItemBasedRecoms.csv',index=False)

In [63]:
recom_df

,recipe_id,recom_ids,recom_names,recom_corrs
0,25775,"[103572, 57772, 49387, 128956, 73033, 90485]","[crock pot black bean salsa chicken oamc, mex...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0]"
1,22526,"[251220, 56846, 84104, 83524, 83506, 21688]","[kittencal s fluffiest scrambled eggs, i hate ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0]"
2,74805,"[34382, 150863, 110683, 3370, 207872, 111734]","[mashed red potatoes with garlic and parmesan,...","[1.0, 1.0, 1.0, 0.9999999999999999, 0.99999999..."
3,83950,"[83950, 30366, 83203, 87782, 155369, 245490]","[real italian bolognese sauce, bev s macaroni...","[1.0, 0.9999999999999999, 0.9999999999999999, ..."
4,155959,"[66241, 136923, 132765, 132261, 129403, 126611]","[buttermilk pancakes, queso blanco dip, microw...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0]"
...,...,...,...,...
2715,158292,"[64235, 29251, 155369, 155018, 87075, 77854]","[blueberry lime jam, jalapeno popper burgers, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0]"
2716,26557,"[22149, 18693, 39618, 41246, 42126, 42564]","[corn on the cob, zucchini chocolate cake, per...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0]"
2717,16702,"[26773, 148513, 37638, 136834, 30484, 146036]","[hot cocoa for one, balsamic tomato couscous, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0]"
2718,34110,"[52366, 53198, 29251, 29210, 106555, 27931]","[outback steakhouse style steak, the ultimate ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0]"
